In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import time
block_size=8
batch_size = 4
max_iters = 1000

In [2]:
start_time=time.time()
print(start_time)

1711800152.865094


In [3]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text=f.read()
chars=sorted(set(text))
print(chars)
vocab_size=len(chars)

['\n', ' ', '!', '#', '$', '%', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™']


In [4]:
print(len(chars))

88


In [5]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long) #This line converts the input text into a tensor of long integers. 
#The encode function is responsible for converting the text into a sequence of numerical tokens.
print(data[:100])

tensor([46, 62, 59,  1, 42, 72, 69, 64, 59, 57, 74,  1, 33, 75, 74, 59, 68, 56,
        59, 72, 61,  1, 59, 28, 69, 69, 65,  1, 69, 60,  1, 46, 62, 59,  1, 49,
        69, 68, 58, 59, 72, 60, 75, 66,  1, 49, 63, 80, 55, 72, 58,  1, 69, 60,
         1, 41, 80,  0,  0,  0, 46, 63, 74, 66, 59, 24,  1, 46, 62, 59,  1, 49,
        69, 68, 58, 59, 72, 60, 75, 66,  1, 49, 63, 80, 55, 72, 58,  1, 69, 60,
         1, 41, 80,  0,  0,  0, 27, 75, 74, 62])


In [6]:
#This sets the context length (or window size) for the bigram model. 
#A bigram model predicts the next token based on the current token and the previous token.


#These lines create the input (x) and target (y) tensors for the first training example. 
#x represents the context (the first block_size tokens), and y represents the targets (the next block_size tokens, shifted by one position).
#x=train_data[:block_size]
#y=train_data[1:block_size+1]

#for t in range(block_size):
   # context=x[:t+1] #This creates a sub-tensor of x representing the context up to the current position t.
   # target=y[t] #This retrieves the target token at position t from y.
   # print('when input is ',context,'target is ', target)
    

In [10]:
print(data)

tensor([46, 62, 59,  ...,  0,  0,  0])


In [7]:
#This line calculates the index at which the dataset should be split into training and validation sets. 
#In this case, 80% of the data is used for training.
n = int(0.8*len(data))
#These lines split the data into training and validation sets based on the index n calculated in the previous step
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[55, 66, 58,  1, 29, 63, 74, 79],
        [75,  1, 55, 72, 59,  1, 63, 68],
        [ 1, 74, 62, 55, 74,  1, 68, 63],
        [72,  0, 55, 73,  1, 74, 62, 59]])
targets:
tensor([[66, 58,  1, 29, 63, 74, 79, 12],
        [ 1, 55, 72, 59,  1, 63, 68,  0],
        [74, 62, 55, 74,  1, 68, 63, 61],
        [ 0, 55, 73,  1, 74, 62, 59,  1]])


In [8]:
end_time=time.time()
lapsed_time=end_time-start_time
print(lapsed_time)

0.025005102157592773


In [13]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)
    


(t-“27dm“VFegJ3y42a.3•!oP;PzBcc1uPbovvEu
(#e R;udVUz3HZ/3I2*,—&($WGui2—N—N[z91jMi6$;cO2QYgJ Qpc*— %•jnJD•fY0;(LBB0X8MvF7e38Rv5y4!sQ2*)t-TkidF3%J1™E58wG—•U(NPJc#G”K’/—HA6vFI7heLilb’l)Xws4S—XA”#J“&C.yxH‘uU*L9[,)f1tl-MPE;8]A#UxOB0gukt!sNvE/k
XBTiI2W7WW;fFG—Bu8X(,-Nu)R.*c0/((dgPeDwH?2)z;(PdiQ.3oUYY4T*%0gl™Z&#
eQ]QPS5EjV*•RC0)O5’R ZTv/Q•RohBYItADt2W(f.37.N—™QDI)‘uTUCkin*3p9#8Zl5bp*f/Gt kg vaRlWPu-;Optw
oB—,oF;#)F
7kir)‘]j pLNfm/oU!”)SajF!g,/—6?%r-d•hY;bH;I?)bSGT[I.4[PsJ)s0gheOcFjeKDC“I.Y3mqKm8B: O
